## Confined aquifer scenario


In [ ]:
from matplotlib import rcParams
rcParams['font.family'] = 'Helvetica'
rcParams['font.size'] = 12
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as mcolors

import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csr_array
from scipy.sparse import save_npz
from scipy.sparse.linalg import inv
from scipy import special as sp
import time

np.set_printoptions(threshold=np.inf)
#%matplotlib widget

In [ ]:
## Poroelastic properties:
G = 10e9 # Shear modulus [GPa]
nu = 0.25 # Drained Poisson's ratio
nu_u = 0.35 # Undrained Poisson's ratio
alpha = 0.9 # Biot coefficient
rho_o = 1000 # Reference water density [kg/m^3]
rho_i = 917 # Ice density [kg/m^3]
g = 10 # gravitational acceleration
k = 1e-13 # Aquifer permeability [m^2]
mu = 1e-3 # Dynamic viscosity of water [kg/(m·s)] 
c_m = alpha*(1-2*nu)/(2*(1-nu)*G) # Uniaxial poroelastic expansion coefficient
gamma = 1/alpha*(nu_u-nu)/((1-2*nu)*(1-nu_u)) #Loading efficiency
c = k*gamma/(mu*c_m) #hydraulic diffusivity [m^2/s]
print(c)
B = 3*(nu_u-nu)/(alpha*(1-2*nu)*(1+nu_u)) # Skempton's coefficient
S = k/(mu*c) # Uniaxial specific storage
S_eps = (1-nu)*(1-2*nu_u)/((1-nu_u)*(1-2*nu))*S #Specific storage at constant strain

# Model domain
L = 10000 # Width of domain [m]
H = 1000 # Height of domain [m]

## Numerical parameters:
dt = 100000 # timestep [s]
nt = 150 # Number of timesteps to compute
times = np.arange(dt,(nt+1)*dt,dt) # Time array
n_plot = 10 #Plot every n_plot time step
dx = 100 # Discretization in x-direction
dy = 50 # Discretization in y-direction
x = np.arange(-L/2,L/2+dx,dx) # x gridpoints
y = np.arange(-H,dy,dy) # y gridpoints
[x2d,y2d] = np.meshgrid(x,y) # x and y mesh
nx = len(x) # Number of gridpoints in x direction
ny = len(y) # Number of gridpoints in y direction

## Growing ice sheet profile: 
retreat = 1000 # in meters
height_drop = 0 # in meters
L_max = L/2
H_max = 1000
L_ice = -retreat*times/(nt*dt)+L_max
H_ice = -height_drop*times/(nt*dt)+H_max
h_ice = np.zeros([nx,nt])

fig1, ax = plt.subplots(2,1,figsize = (5,5),constrained_layout=True)
cmap = plt.cm.viridis_r 
colors = cmap(np.linspace(0, 1, nt))
gl_index = np.zeros((nt,))

for n in np.arange(0,nt):
    for i in np.arange(0,nx): 
        if (1-((x[i]+L/2)/(L_ice[n]))**2)>0:
            h_ice[i,n] = H_ice[n]*np.sqrt(1-((x[i]+L/2)/(L_ice[n]))**2)
    gl_index[n] = np.where(h_ice[:,n] == 0)[0][0]
    if n % n_plot==0:
        ax[0].plot(x/1e3,h_ice[:,n],'.-',color=colors[n])
        ax[0].set_xlabel('x [km]')
        ax[0].set_ylabel('$h_{ice}$ [m]')
        
        ax[1].plot(x/1e3,h_ice[:,n]-h_ice[:,0],'.-',color=colors[n])
        ax[1].set_xlabel('x [km]')
        ax[1].set_ylabel('$\Delta h_{ice}$ [m]')

sigma_surf = -rho_i*g*(h_ice-h_ice[:,0][:,None])

# sigma_o = 1e6 # Amplitude of applied stress [Pa]
# sigma_load = -sigma_o/times[nt//2-1]*times #Stress at the surface (negative = compressive loading)
# sigma_load[nt//2:nt] = -sigma_o

# sigma_surf = np.zeros([nx,nt])
# sigma_surf[:nx//4,:] = sigma_load

p_surf = np.zeros([nx,nt])
#p_surf = -sigma_surf*0.9

tau_surf = np.zeros([nx,nt])
#tau_surf[:nx//4,:] = sigma_load

## Setup arrays for variables to be solved for:
p = np.zeros([nx*ny,1]) # Initial pore pressure [Pa]
u = np.zeros([nx*ny,1]) # Initial horizontal displacements [m]
v = np.zeros([nx*ny,1]) # Initial vertical displacements [m]

## Array of indices:
Number = np.arange(0,nx*ny) 
Number = Number.reshape((nx,ny))


In [ ]:
# Construct A matrix to be inverted:

n_inv = retreat//dx+1 # Number of inverses to take

A_inv_list = []

# Compute inverse of matrix A: 
start_time = time.time()

for n in np.arange(0,n_inv+1):
    A1_p = np.zeros((nx*ny,nx*ny)) #Pressure portion of fluid equation
    A1_u = np.zeros((nx*ny,nx*ny)) #x-displacement portion of fluid equation
    A1_v = np.zeros((nx*ny,nx*ny)) #y-displacement portion of fluid equation

    A2_p = np.zeros((nx*ny,nx*ny)) #Pressure portion of first mechanical equation
    A2_u = np.zeros((nx*ny,nx*ny)) #x-displacement portion of first mechanical equation
    A2_v = np.zeros((nx*ny,nx*ny)) #y-displacement portion of first mechanical equation

    A3_p = np.zeros((nx*ny,nx*ny)) #Pressure portion of second mechanical equation
    A3_u = np.zeros((nx*ny,nx*ny)) #x-displacement portion of second mechanical equation
    A3_v = np.zeros((nx*ny,nx*ny)) #y-displacement portion of second mechanical equation

    for i in np.arange(0,nx):
        for j in np.arange(0,ny):   

            # Top boundary:
            if j == ny-1: 
                #A1_p[Number[i,j],Number[i,j]] = 1 # p = 0 everywhere at top boundary

                # Upper left corner:
                if i == 0:
                    A1_p[Number[i,j],Number[i+1,j]] = -2*(k/(alpha*mu))/dx**2
                    #A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2 #p_{i+1,j} - p_{i-1,j} = 0
                    A1_p[Number[i,j],Number[i,j-1]] = -2*(k/(alpha*mu))/dy**2
                    A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)+(1-2*nu)/(1-nu)*alpha/(2*G*dt)
                    A1_u[Number[i,j],Number[i+1,j]] = 2*(1-2*nu)/(1-nu)*1/(2*dt*dx)
                    #A1_u[Number[i,j],Number[i-1,j]] = -(1-2*nu)/(1-nu)*1/(2*dt*dx) #u_{i-1,j} = -u_{i+1,j}

                    #A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G) #p_{i+1,j} - p_{i-1,j} = 0
                    #A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                    A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))+nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    #A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                    #A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2) #u_{i+1,j} = -u_{i-1,j}
                    #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                    A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                    A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    #A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)) # u_{i-2,j} = -u_{i,j}
                    #A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy)
                    #A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy) #v_{i+1,j} - v_{i-1,j} = 0

                   #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G) #p_{i,j+1} - p_{i,j-1} = 0
                   #A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G) #p_{i,j+1} - p_{i,j-1} = 0
                    A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                    A3_u[Number[i,j],Number[i+1,j]] = -4*nu/((1-2*nu)*dx*dy) #u_{i+1,j} = -u_{i-1,j}
                    #A3_u[Number[i,j],Number[i-1,j]] = 2*nu/((1-2*nu)*dx*dy) 
                    A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                    A3_v[Number[i,j],Number[i+1,j]] = 2/(dx**2) #v_{i+1,j} - v_{i-1,j} = 0
                    #A3_v[Number[i,j],Number[i-1,j]] = 1/(dx**2)
                    A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                    #A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)-1/((1-2*nu)*4*dx**2)  #v_{i,j} - v_{i-2,j} = 0  

                # Gridpoint to the right of upper left corner (necessary because of the i-2 term):
                elif i == 1:
                    A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2
                    A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
                    A1_p[Number[i,j],Number[i,j-1]] = -2*(k/(alpha*mu))/dy**2
                    A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)+(1-2*nu)/(1-nu)*alpha/(2*G*dt)
                    A1_u[Number[i,j],Number[i+1,j]] = (1-2*nu)/(1-nu)*1/(2*dt*dx)
                    A1_u[Number[i,j],Number[i-1,j]] = -(1-2*nu)/(1-nu)*1/(2*dt*dx)

                    A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                    A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                    A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))+nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                    A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                    #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                    A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                    A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    #A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)) # u_{i-2,j} = -u_{i,j}
                    A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy)
                    A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy)  

                   #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G) #p_{i,j+1} - p_{i,j-1} = 0
                   #A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G) #p_{i,j+1} - p_{i,j-1} = 0
                    A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                    A3_u[Number[i,j],Number[i+1,j]] = -2*nu/((1-2*nu)*dx*dy)
                    A3_u[Number[i,j],Number[i-1,j]] = 2*nu/((1-2*nu)*dx*dy)
                    A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                    A3_v[Number[i,j],Number[i+1,j]] = 1/(dx**2)
                    A3_v[Number[i,j],Number[i-1,j]] = 1/(dx**2)
                    A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                    #A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)-1/((1-2*nu)*4*dx**2)  #v_{i,j} - v_{i-2,j} = 0                         

                # Rest of top boundary:
                elif i<=nx/2-n:
                    A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2
                    A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
                    A1_p[Number[i,j],Number[i,j-1]] = -2*(k/(alpha*mu))/dy**2
                    A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)+(1-2*nu)/(1-nu)*alpha/(2*G*dt)
                    A1_u[Number[i,j],Number[i+1,j]] = (1-2*nu)/(1-nu)*1/(2*dt*dx)
                    A1_u[Number[i,j],Number[i-1,j]] = -(1-2*nu)/(1-nu)*1/(2*dt*dx)

                    A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                    A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                    A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))
                    A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                    A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                    #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                    A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                    A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy)
                    A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy)

                   #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G) #p_{i,j+1} - p_{i,j-1} = 0
                   #A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G) #p_{i,j+1} - p_{i,j-1} = 0
                    A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                    A3_u[Number[i,j],Number[i+1,j]] = -2*nu/((1-2*nu)*dx*dy)
                    A3_u[Number[i,j],Number[i-1,j]] = 2*nu/((1-2*nu)*dx*dy)
                    A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                    A3_v[Number[i,j],Number[i+1,j]] = 1/(dx**2)
                    A3_v[Number[i,j],Number[i-1,j]] = 1/(dx**2)
                    A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)

                # Gridpoint to the left of upper right corner (necessary because of the i+2 term):
                elif i == nx-2:
                    A1_p[Number[i,j],Number[i,j]] = 1

                    A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                    A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                    A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))+nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                    A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                    #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                    A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                    #A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)) # u_{i+2,j} = -u_{i,j}
                    A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy) 
                    A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy)

                   #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)  #p_{i,j+1} = - p_{i,j-1}
                    A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G)
                    A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                    A3_u[Number[i,j],Number[i+1,j]] = -2*nu/((1-2*nu)*dx*dy)
                    A3_u[Number[i,j],Number[i-1,j]] = 2*nu/((1-2*nu)*dx*dy)
                    A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                    A3_v[Number[i,j],Number[i+1,j]] = 1/(dx**2)
                    A3_v[Number[i,j],Number[i-1,j]] = 1/(dx**2)
                    #A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)-1/((1-2*nu)*4*dx**2) #v_{i+2,j} - v_{i,j} = 0

                # Upper right corner:        
                elif i == nx-1:    
                    A1_p[Number[i,j],Number[i,j]] = 1

                    #A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G) #p_{i+1,j} - p_{i-1,j} = 0
                    #A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)           
                    A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))+nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    #A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2) #u_{i+1,j} = -u_{i-1,j}
                    #A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2) #u_{i+1,j} = -u_{i-1,j}
                    #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                    A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                    #A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)) # u_{i+2,j} = -u_{i,j}
                    A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    #A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy) #v_{i+1,j} - v_{i-1,j} = 0
                    #A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy)

                   #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)  #p_{i,j+1} = - p_{i,j-1}
                    A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G)
                    A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                    #A3_u[Number[i,j],Number[i+1,j]] = -2*nu/((1-2*nu)*dx*dy)
                    A3_u[Number[i,j],Number[i-1,j]] = 4*nu/((1-2*nu)*dx*dy) #u_{i+1,j} = -u_{i-1,j}
                    A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                    #A3_v[Number[i,j],Number[i+1,j]] = 1/(dx**2)
                    A3_v[Number[i,j],Number[i-1,j]] = 2/(dx**2) #v_{i+1,j} - v_{i-1,j} = 0
                    #A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)-1/((1-2*nu)*4*dx**2) #v_{i+2,j} - v_{i,j} = 0

                # Rest of top boundary:
                else:
                    A1_p[Number[i,j],Number[i,j]] = 1

                    A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                    A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                    A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))
                    A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                    A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                    #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                    A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                    A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                    A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy)
                    A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy)

                   #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G) #p_{i,j+1} = - p_{i,j-1}
                    A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G)
                    A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                    A3_u[Number[i,j],Number[i+1,j]] = -2*nu/((1-2*nu)*dx*dy)
                    A3_u[Number[i,j],Number[i-1,j]] = 2*nu/((1-2*nu)*dx*dy)
                    A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                    A3_v[Number[i,j],Number[i+1,j]] = 1/(dx**2)
                    A3_v[Number[i,j],Number[i-1,j]] = 1/(dx**2)
                    A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                    A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)

            # Bottom boundary:
            elif j == 0:

                A2_u[Number[i,j],Number[i,j]] = 1
                A3_v[Number[i,j],Number[i,j]] = 1

                # Bottom left corner: 
                if i == 0:
                    A1_p[Number[i,j],Number[i+1,j]] = -2*(k/(alpha*mu))/dx**2  #p_{i+1,j} - p_{i-1,j} = 0
                    #A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
                    A1_p[Number[i,j],Number[i,j+1]] = -2*(k/(alpha*mu))/dy**2 #p_{i,j+1} - p_{i,j-1} = 0
                    #A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
                    A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
                    A1_u[Number[i,j],Number[i+1,j]] = 2/(2*dt*dx) #u_{i+1,j} = -u_{i-1,j}
                    #A1_u[Number[i,j],Number[i-1,j]] = -1/(2*dt*dx)
                    A1_v[Number[i,j],Number[i,j+1]] = 2/(2*dt*dy) #v_{i,j+1} = -v_{i-1,j}
                    #A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)   

                # Bottom right corner:
                elif i == nx-1:
                    #A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2
                    A1_p[Number[i,j],Number[i-1,j]] = -2*(k/(alpha*mu))/dx**2  #p_{i+1,j} - p_{i-1,j} = 0
                    A1_p[Number[i,j],Number[i,j+1]] = -2*(k/(alpha*mu))/dy**2 #p_{i,j+1} - p_{i,j-1} = 0
                    #A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
                    A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
                    #A1_u[Number[i,j],Number[i+1,j]] = 1/(2*dt*dx) #u_{i+1,j} = -u_{i-1,j}
                    A1_u[Number[i,j],Number[i-1,j]] = -2/(2*dt*dx)
                    A1_v[Number[i,j],Number[i,j+1]] = 2/(2*dt*dy) #v_{i,j+1} = -v_{i-1,j}
                    #A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)   

                # Rest of bottom boundary:    
                else: 
                    A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2
                    A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
                    A1_p[Number[i,j],Number[i,j+1]] = -2*(k/(alpha*mu))/dy**2 #p_{i,j+1} - p_{i,j-1} = 0
                    #A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2 
                    A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
                    A1_u[Number[i,j],Number[i+1,j]] = 1/(2*dt*dx)
                    A1_u[Number[i,j],Number[i-1,j]] = -1/(2*dt*dx)
                    A1_v[Number[i,j],Number[i,j+1]] = 2/(2*dt*dy) #v_{i,j+1} = -v_{i-1,j}
                    #A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)      

            # Left boundary:
            elif i == 0: 
                A1_p[Number[i,j],Number[i+1,j]] = -2*(k/(alpha*mu))/dx**2 #p_{i+1,j} - p_{i-1,j} = 0
                #A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
                A1_p[Number[i,j],Number[i,j+1]] = -(k/(alpha*mu))/dy**2
                A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
                A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
                A1_u[Number[i,j],Number[i+1,j]] = 2/(2*dt*dx)  #u_{i+1,j} = -u_{i-1,j}
                #A1_u[Number[i,j],Number[i-1,j]] = -1/(2*dt*dx)
                A1_v[Number[i,j],Number[i,j+1]] = 1/(2*dt*dy)
                A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)   

                A2_u[Number[i,j],Number[i,j]] = 1

                A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)
                A3_p[Number[i,j],Number[i,j-1]] = alpha/(2*dy*G)
                A3_v[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dy**2)+1/dx**2)
                A3_v[Number[i,j],Number[i+1,j]] = 2/dx**2 #v_{i+1,j} - v_{i-1,j} = 0
                #A3_v[Number[i,j],Number[i-1,j]] = 1/dx**2
                A3_v[Number[i,j],Number[i,j+1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
                A3_v[Number[i,j],Number[i,j-1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
                A3_u[Number[i,j],Number[i+1,j+1]] = 2/(1-2*nu)*(1/(4*dx*dy)) #u_{i+1,j} = -u_{i-1,j}
                A3_u[Number[i,j],Number[i+1,j-1]] = -2/(1-2*nu)*(1/(4*dx*dy)) #u_{i+1,j} = -u_{i-1,j}
                #A3_u[Number[i,j],Number[i-1,j+1]] = -1/(1-2*nu)*(1/(4*dx*dy))
                #A3_u[Number[i,j],Number[i-1,j-1]] = 1/(1-2*nu)*(1/(4*dx*dy))   

            # Right boundary:
            elif i == nx-1:
                #A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2 
                A1_p[Number[i,j],Number[i-1,j]] = -2*(k/(alpha*mu))/dx**2 #p_{i+1,j} - p_{i-1,j} = 0
                A1_p[Number[i,j],Number[i,j+1]] = -(k/(alpha*mu))/dy**2
                A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
                A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
                #A1_u[Number[i,j],Number[i+1,j]] = 1/(2*dt*dx) 
                A1_u[Number[i,j],Number[i-1,j]] = -2/(2*dt*dx)  #u_{i+1,j} = -u_{i-1,j}
                A1_v[Number[i,j],Number[i,j+1]] = 1/(2*dt*dy)
                A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)      

                A2_u[Number[i,j],Number[i,j]] = 1
                A3_v[Number[i,j],Number[i,j]] = 1

                # A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)
                # A3_p[Number[i,j],Number[i,j-1]] = alpha/(2*dy*G)
                # A3_v[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dy**2)+1/dx**2)
                # #A3_v[Number[i,j],Number[i+1,j]] = 1/dx**2
                # A3_v[Number[i,j],Number[i-1,j]] = 2/dx**2 #v_{i+1,j} - v_{i-1,j} = 0
                # A3_v[Number[i,j],Number[i,j+1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
                # A3_v[Number[i,j],Number[i,j-1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
                # #A3_u[Number[i,j],Number[i+1,j+1]] = 1/(1-2*nu)*(1/(4*dx*dy))
                # #A3_u[Number[i,j],Number[i+1,j-1]] = -1/(1-2*nu)*(1/(4*dx*dy))
                # A3_u[Number[i,j],Number[i-1,j+1]] = -2/(1-2*nu)*(1/(4*dx*dy)) #u_{i+1,j} = -u_{i-1,j}
                # A3_u[Number[i,j],Number[i-1,j-1]] = 2/(1-2*nu)*(1/(4*dx*dy))  #u_{i+1,j} = -u_{i-1,j}

            # Domain interior: 
            else: 
                A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2
                A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
                A1_p[Number[i,j],Number[i,j+1]] = -(k/(alpha*mu))/dy**2
                A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
                A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
                A1_u[Number[i,j],Number[i+1,j]] = 1/(2*dt*dx)
                A1_u[Number[i,j],Number[i-1,j]] = -1/(2*dt*dx)
                A1_v[Number[i,j],Number[i,j+1]] = 1/(2*dt*dy)
                A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)     

                A2_p[Number[i,j],Number[i+1,j]] = -alpha/(2*dx*G)
                A2_p[Number[i,j],Number[i-1,j]] = alpha/(2*dx*G)
                A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2)
                A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                A2_u[Number[i,j],Number[i,j-1]] = 1/dy**2
                A2_v[Number[i,j],Number[i+1,j+1]] = 1/(1-2*nu)*(1/(4*dx*dy))
                A2_v[Number[i,j],Number[i+1,j-1]] = -1/(1-2*nu)*(1/(4*dx*dy))
                A2_v[Number[i,j],Number[i-1,j+1]] = -1/(1-2*nu)*(1/(4*dx*dy))
                A2_v[Number[i,j],Number[i-1,j-1]] = 1/(1-2*nu)*(1/(4*dx*dy))

                A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)
                A3_p[Number[i,j],Number[i,j-1]] = alpha/(2*dy*G)
                A3_v[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dy**2)+1/dx**2)
                A3_v[Number[i,j],Number[i+1,j]] = 1/dx**2
                A3_v[Number[i,j],Number[i-1,j]] = 1/dx**2
                A3_v[Number[i,j],Number[i,j+1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
                A3_v[Number[i,j],Number[i,j-1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
                A3_u[Number[i,j],Number[i+1,j+1]] = 1/(1-2*nu)*(1/(4*dx*dy))
                A3_u[Number[i,j],Number[i+1,j-1]] = -1/(1-2*nu)*(1/(4*dx*dy))
                A3_u[Number[i,j],Number[i-1,j+1]] = -1/(1-2*nu)*(1/(4*dx*dy))
                A3_u[Number[i,j],Number[i-1,j-1]] = 1/(1-2*nu)*(1/(4*dx*dy))   

    # Concatenate the submatrices into one big matrix to be inverted:            
    A1 = np.concatenate((A1_p, A1_u, A1_v), axis=1)
    A2 = np.concatenate((A2_p, A2_u, A2_v), axis=1)
    A3 = np.concatenate((A3_p, A3_u, A3_v), axis=1)
    A = np.concatenate((A1, A2, A3),axis=0)

    A_sparse = csr_matrix(A)
    A_inv = inv(A_sparse) 
    
    A_inv_list.append(A_inv)
    
time_inv = time.time() - start_time

print(A_inv_list[0].shape)

print('Matrix A took ' + str(time_inv) + ' s to invert')

In [ ]:
### Time loop to compute solution m given the linear system of equations A*m = b at each time step:

## Initialize figures:
fig2, ax2 = plt.subplots(1,4,figsize = (15,4),constrained_layout=True)
fig3, ax3 = plt.subplots(1,2,figsize = (7,4),constrained_layout=True)

## Main time loop, plotting results every n_plot timestep: 
for n in np.arange(0,nt-1):    
      
    # Initial b-vectors containing the RHS of each equation:
    
    b1 = np.zeros([nx*ny,1]) # b vector for fluid equation
    b2 = np.zeros([nx*ny,1]) # b vector for first mechanical equation
    b3 = np.zeros([nx*ny,1]) # b vector for second mechanical equation

    for i in np.arange(0,nx):
        for j in np.arange(0,ny):   

            # Top boundary:
            if j == ny-1:                 
                if i == 0:
                    b1[Number[i,j]] = 1/(2*dx*dt)*(1-2*nu)/(1-nu)*(2*u[Number[i+1,j]])+(S_eps/(alpha*dt)+(1-2*nu)/(1-nu)*alpha/(2*G*dt))*p[Number[i,j]]+(1-2*nu)/(1-nu)*1/(2*G*dt)*(sigma_surf[i,n]-sigma_surf[i,n+1])
                    b2[Number[i,j]] = -2*tau_surf[i,n]/(G*dy)
                    b3[Number[i,j]] = -2*sigma_surf[i,n]/(G*dy)
                elif i<=gl_index[n]:
                    b1[Number[i,j]] = 1/(2*dx*dt)*(1-2*nu)/(1-nu)*(u[Number[i+1,j]]-u[Number[i-1,j]])+(S_eps/(alpha*dt)+(1-2*nu)/(1-nu)*alpha/(2*G*dt))*p[Number[i,j]]+(1-2*nu)/(1-nu)*1/(2*G*dt)*(sigma_surf[i,n]-sigma_surf[i,n+1])
                    b2[Number[i,j]] = -2*tau_surf[i,n]/(G*dy)-(1/(2*G*(1-nu)*2*dx))*(sigma_surf[i+1,n]-sigma_surf[i-1,n])
                    b3[Number[i,j]] = -2*sigma_surf[i,n]/(G*dy)-(1/(G*(1-2*nu)*2*dx))*(tau_surf[i+1,n]-tau_surf[i-1,n])
                elif i == nx-1:
                    b1[Number[i,j]] = p_surf[i,n]
                    b2[Number[i,j]] = -2*tau_surf[i,n]/(G*dy)
                    b3[Number[i,j]] = -2*sigma_surf[i,n]/(G*dy)
                else:
                    b1[Number[i,j]] = p_surf[i,n]
                    b2[Number[i,j]] = -2*tau_surf[i,n]/(G*dy)-(1/(2*G*(1-nu)*2*dx))*(sigma_surf[i+1,n]-sigma_surf[i-1,n])
                    b3[Number[i,j]] = -2*sigma_surf[i,n]/(G*dy)-(1/(G*(1-2*nu)*2*dx))*(tau_surf[i+1,n]-tau_surf[i-1,n])
            
            # Bottom boundary:
            elif j == 0:
                b2[Number[i,j]] = 0
                b3[Number[i,j]] = 0
                
                # Bottom left corner:
                if i == 0:
                     b1[Number[i,j]] = 2/(2*dx*dt)*(u[Number[i+1,j]])+2/(2*dy*dt)*(v[Number[i,j+1]])+S_eps/(alpha*dt)*p[Number[i,j]]  
                
                # Bottom right corner:
                elif i == nx-1:
                     b1[Number[i,j]] = 2/(2*dx*dt)*(-u[Number[i-1,j]])+2/(2*dy*dt)*(v[Number[i,j+1]])+S_eps/(alpha*dt)*p[Number[i,j]]  

                # Rest of bottom boundary:        
                else: 
                     b1[Number[i,j]] = 1/(2*dx*dt)*(u[Number[i+1,j]]-u[Number[i-1,j]])+2/(2*dy*dt)*(v[Number[i,j+1]])+S_eps/(alpha*dt)*p[Number[i,j]]  

            # Left boundary:            
            elif i == 0: 
                b1[Number[i,j]] = 1/(2*dx*dt)*(2*u[Number[i+1,j]])+1/(2*dy*dt)*(v[Number[i,j+1]]-v[Number[i,j-1]])+S_eps/(alpha*dt)*p[Number[i,j]]
                b2[Number[i,j]] = 0
                b3[Number[i,j]] = 0
           
            # Right boundary
            elif i == nx-1:              
                b1[Number[i,j]] = 1/(2*dx*dt)*(-2*u[Number[i-1,j]])+1/(2*dy*dt)*(v[Number[i,j+1]]-v[Number[i,j-1]])+S_eps/(alpha*dt)*p[Number[i,j]]              
                b2[Number[i,j]] = 0
                b3[Number[i,j]] = 0
                
            # Domain interior: 
            else: 
                b1[Number[i,j]] = 1/(2*dx*dt)*(u[Number[i+1,j]]-u[Number[i-1,j]])+1/(2*dy*dt)*(v[Number[i,j+1]]-v[Number[i,j-1]])+S_eps/(alpha*dt)*p[Number[i,j]]
                b2[Number[i,j]] = 0
                b3[Number[i,j]] = 0    

    b = np.concatenate((b1, b2, b3),axis=0)
    
    # Solve for solution vector m:
    m_sol = A_inv_list[int(nx/2-gl_index[n])]*b
        
    # Update p, u, and v: 
    p = m_sol[Number.flatten()]
    u = m_sol[Number.flatten()+nx*ny]
    v = m_sol[Number.flatten()+2*nx*ny]
    
    p_array = p.reshape((nx,ny))
    v_array = v.reshape((nx,ny))
    u_array = u.reshape((nx,ny))
    
    # Compute strains & stress:
    eps_xx = np.zeros((nx*ny,1))
    eps_yy = np.zeros((nx*ny,1))
    eps_xy = np.zeros((nx*ny,1))
    eps_kk =  np.zeros((nx*ny,1))
    sigma_xx = np.zeros((nx*ny,1))
    sigma_yy = np.zeros((nx*ny,1))
    sigma_xy = np.zeros((nx*ny,1))
    
    q_aq = np.zeros((nx,1))
    
    for i in np.arange(0,nx):
        q_aq[i] = -(k/mu)*(p[Number[i,ny-1]]-p[Number[i,ny-2]])/dy
    
    for i in np.arange(1,nx-1):
        for j in np.arange(1,ny-1):   
            eps_xx[Number[i,j]] = (u[Number[i+1,j]]-u[Number[i+1,j]])/(2*dx)
            eps_yy[Number[i,j]] = (v[Number[i,j+1]]-v[Number[i,j-1]])/(2*dy)
            eps_xy[Number[i,j]] = 0.5*(u[Number[i,j+1]]-u[Number[i,j-1]])/(2*dy)+0.5*(v[Number[i+1,j]]-v[Number[i-1,j]])/(2*dx)
            eps_kk[Number[i,j]] = eps_xx[Number[i,j]]+eps_yy[Number[i,j]]
            sigma_xx[Number[i,j]] = 2*G*eps_xx[Number[i,j]]+2*G*nu/(1-2*nu)*eps_kk[Number[i,j]]-alpha*p[Number[i,j]]
            sigma_yy[Number[i,j]] = 2*G*eps_yy[Number[i,j]]+2*G*nu/(1-2*nu)*eps_kk[Number[i,j]]-alpha*p[Number[i,j]]
            sigma_xy[Number[i,j]] = 2*G*eps_xy[Number[i,j]]
            
    sigma_yy = sigma_yy.reshape((nx,ny))
    sigma_xx = sigma_xx.reshape((nx,ny))
    sigma_xy = sigma_xy.reshape((nx,ny))
 
    if n % n_plot == 0:
        fig, ax = plt.subplots(1,3,figsize = (15,4),constrained_layout=True)
        p1 = ax[0].pcolor(x2d/1e3,y2d/1e3,p_array.T/1e6,cmap='RdBu',shading = 'auto',vmin=-0.15,vmax=0.15)
        ax[0].contour(x2d/1e3, y2d/1e3, p_array.T/1e6, levels=np.arange(-0.15,0.15,0.01),colors='k',linewidths=0.5)
        ax[0].plot(x/1e3,h_ice[:,1]/1e3,':k')
        ax[0].plot(x/1e3,h_ice[:,n]/1e3,'-k')
        ax[0].set_xlabel("x [km]")
        ax[0].set_ylabel("y [km]")
        #ax[0].set_title('Time = '+str(round(time/3600/24))+' days')
        cb = plt.colorbar(p1,ax=ax[0],aspect = 20)
        cb.set_label(label='$p$ [MPa]')
        #cb.ax.tick_params(labelsize=16)
        
        p2 = ax[1].pcolor(x2d/1e3,y2d/1e3,v_array.T*1e3,cmap='RdBu',shading = 'auto',vmin=-150,vmax=150)
        ax[1].contour(x2d/1e3, y2d/1e3, v_array.T*1e3, levels=np.arange(-150,150,10),colors='k',linewidths=0.5)
        ax[1].plot(x/1e3,h_ice[:,1]/1e3,':k')
        ax[1].plot(x/1e3,h_ice[:,n]/1e3,'-k')
        ax[1].set_xlabel("x [km]")
        ax[1].set_ylabel("y [km]")
        #ax[0].set_title('Time = '+str(round(time/3600/24))+' days')
        cb = plt.colorbar(p2,ax=ax[1],aspect = 20)
        cb.set_label(label='$v$ [mm]')
        #cb.ax.tick_params(labelsize=16)

        p3 = ax[2].pcolor(x2d/1e3,y2d/1e3,sigma_yy.T/1e6,cmap='RdBu',shading = 'auto',vmin=-10,vmax=10)
        ax[2].contour(x2d/1e3, y2d/1e3, sigma_yy.T/1e6, levels=np.arange(-10,10,1),colors='k',linewidths=0.5)
        ax[2].plot(x/1e3,h_ice[:,1]/1e3,':k')
        ax[2].plot(x/1e3,h_ice[:,n]/1e3,'-k')
        ax[2].set_xlabel("x [km]")
        ax[2].set_ylabel("y [km]")
        #ax[0].set_title('Time = '+str(round(time/3600/24))+' days')
        cb = plt.colorbar(p3,ax=ax[2],aspect = 20)
        cb.set_label(label='$\sigma_{yy}$ [MPa]')
        #cb.ax.tick_params(labelsize=16)
        
        ax2[0].plot(p_array[nx//4,:]/1e6,y,'.-',color=colors[n])
        ax2[0].set_xlabel("p [MPa]")
        ax2[0].set_ylabel("y [km]")
        ax2[0].set_title("Pore pressure")
        
        ax2[1].plot(x/1e3,v_array[:,ny-1]*1e3,'.-',color=colors[n])
        ax2[1].set_xlabel("x [km]")
        ax2[1].set_ylabel("v [mm]")
        ax2[1].set_title("Vertical displacement")
        
        ax2[2].plot(x/1e3,u_array[:,ny-1]*1e3,'.-',color=colors[n])
        ax2[2].set_xlabel("x [km]")
        ax2[2].set_ylabel("u [mm]")
        ax2[2].set_title("Horizontal displacement")
        
        ax2[3].plot(sigma_yy[nx//4,1:ny-2]/1e6,y[1:ny-2],'.-',color=colors[n])
        ax2[3].set_xlabel("$\sigma_{yy}$ [MPa]")
        ax2[3].set_ylabel("y[km]")
        ax2[3].set_title("Vertical stress")
        
        ax3[0].plot(x/1e3,p_array[:,ny-1]/1e6,'.-',color=colors[n])
        ax3[0].set_xlabel("x [km]")
        ax3[0].set_ylabel("$\Delta p$ [MPa]")
        ax3[0].set_title("Pore pressure")
        #ax3[2].set_ylim(-5e-6,7e-6)
        
        ax3[1].plot(x/1e3,q_aq,'.-',color=colors[n])
        ax3[1].set_xlabel("x [km]")
        ax3[1].set_ylabel("$q_{aq}$ [m/s]")
        ax3[1].set_title("Specific discharge")
        #ax3[1].set_ylim(-5e-6,7e-6)
